In [3]:
import logging
import time
import random
from datetime import datetime

n_campi = 'CAFI'

data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

logger = logging.getLogger('consolidação')
logger.setLevel(logging.DEBUG)

fHandler = logging.FileHandler(f'logs/{data_e_hora_atuais}_consolidacao.log', mode='w')
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fHandler.setFormatter(formatter)
logger.addHandler(fHandler)

# logging.basicConfig(level=logging.DEBUG, filename='app.log', filemode='x', format='%(asctime)s - %(levelname)s - %(message)s')
logger.info(f'Iniciando consolidação campus {n_campi}')
# logger.debug('debug message')
# logger.warning('warn message')

In [4]:
import pandas as pd

logger.info(f'carregando palnilha de levantamento fisico campus {n_campi}')
levantamento = pd.read_excel('consolidacao/levantamento/2023_levantamento_CAFI.xlsx')

logger.info(f'carregando palnilha de estoque suap campus {n_campi}')
suap = pd.read_excel('consolidacao/suap/2023-11-28_17-08_CAFI_relatorio_almox.xlsx')

logger.info(f'tratando dados da planilha de levantameto fisico {n_campi}')

# removendo itens nulos e deixando apenas os retificados
retificacao = levantamento[levantamento['Retificação'].str.contains('Retificação').fillna(False)]


# lista de itens vencidos
# vencido = levantamento[levantamento['Material Vencido - MV'].str.contains('x').fillna(False)]
# lista de itens não identificados
# nao_identificado = levantamento[levantamento['Sem identificação - SI'].str.contains('x').fillna(False)]

# print(levantamento.info())
# print(suap.info())

# planilha contendo apenas itens válidos
levantamento_ret = levantamento.drop_duplicates(subset=['Código'], keep='last')
levantamento_ret = levantamento_ret.rename(columns={'Código': 'codigo'})

# elemento despesa do levantamento
ed_lev = levantamento_ret['codigo'].str[3:5]


# adicionando ed ao data frame geral
# levantamento_ret['ED'] = ed_lev

logger.info(f'removendo dados duplicados')
# lista de servidores
servidores = levantamento['Endereço de e-mail'].drop_duplicates(keep='last')
# reindexando uutilizando list comprehension
servidores.index = [x for x in range(0, len(servidores))]

logger.info(f'cruzando dados entre levantamento fisico e suap')

levantamento_cross = levantamento_ret.drop(columns=['Material de pouca movimentação - PM', 'Material Vencido - MV',
                                                 'Material próximo ao vencimento - PV', 'Material danificado ou avariado - MD',
                                                 'Material Obsoleto - MO', 'Sem identificação - SI', 'Valor unitário',
                                                 'Campus', 'Retificação', 'Carimbo de data/hora'])
levantamento_cross = levantamento_cross.rename(columns={'Endereço de e-mail': 'servidor', 'Nome': 'descricao_lev',
                                                        'Quantidade': 'qtd_lev'})
levantamento_cross = levantamento_cross[['codigo', 'descricao_lev', 'qtd_lev', 'servidor']]

suap_cross = suap.rename(columns={'descricao': 'descricao_suap', 'qtd': 'qtd_suap'})
suap_cross = suap_cross.drop(columns=['vlr_total', 'vlr_unt'])
# display(suap_cross)
# display(levantamento_cross)

logger.info(f'consolidando dados')

logger.info(f'exportando planilha de consolidação')
consolidacao = pd.merge(suap_cross, levantamento_cross, on='codigo', how='outer', indicator=True)
consolidacao.to_excel(f'consolidacao/{n_campi}_consolidacao.xlsx', index=False)

logger.info(f'consolidação campus {n_campi} finalizada')

In [5]:
logger.info(f'montando planilha de ajustes ')

consolidacao['diferenca'] = consolidacao['qtd_suap'] - consolidacao['qtd_lev']
display(consolidacao)

pedidos = consolidacao[consolidacao['_merge'].str.contains('both')]
pedidos = pedidos[['codigo','ed' ,'descricao_suap', 'qtd_suap', 'qtd_lev', 'diferenca']]

acao = []
for valor in pedidos['diferenca']:
    
    if valor > 0:
        baixa = "pedido no suap"
        acao.append(baixa)
    elif valor < 0:
        baixa = "saida sem pedido"
        acao.append(baixa)
    else:
        baixa = 'adequado'
        acao.append(baixa)

logger.info(f'exportando planilha de ajustes ')

pedidos['acao'] = acao
pedidos.to_excel(f'consolidacao/pedidos/{n_campi}_pedidos.xlsx', index=False)
display(pedidos)

,codigo,descricao_suap,und,qtd_suap,ed,descricao_lev,qtd_lev,servidor,_merge,diferenca
0,30.14.117,"APITO, MATERIAL PLÁSTICO, APLICAÇÃO ESPORTE, T...",UN,11.0,14 - MATERIAL EDUCATIVO E ESPORTIVO,"APITO, MATERIAL PLÁSTICO, APLICAÇÃO ESPORTE, T...",11,ricardo.silva@afogados.ifpe.edu.br,both,0.0
1,30.14.121,REDE ESPORTE BASQUETE,UN,7.0,14 - MATERIAL EDUCATIVO E ESPORTIVO,REDE ESPORTE BASQUETE,7,ricardo.silva@afogados.ifpe.edu.br,both,0.0
2,30.14.122,REDE ESPORTE FUTSAL,UN,10.0,14 - MATERIAL EDUCATIVO E ESPORTIVO,REDE ESPORTE FUTSAL,10,ricardo.silva@afogados.ifpe.edu.br,both,0.0
3,30.14.123,REDE ESPORTE FUTEBOL DE CAMPO,UN,3.0,14 - MATERIAL EDUCATIVO E ESPORTIVO,REDE ESPORTE FUTEBOL DE CAMPO,3,ricardo.silva@afogados.ifpe.edu.br,both,0.0
4,30.14.13,BOLA DE FUTEBOL DE CAMPO,UN,5.0,14 - MATERIAL EDUCATIVO E ESPORTIVO,BOLA DE FUTEBOL DE CAMPO,13,ricardo.silva@afogados.ifpe.edu.br,both,-8.0
...,...,...,...,...,...,...,...,...,...,...
477,30.16.1175,NaN,NaN,NaN,NaN,"REFIL PARA PINCEL DE QUADRO BRANCO, 5,5 ML, CO...",0,ricardo.silva@afogados.ifpe.edu.br,right_only,NaN
478,30.16.426,NaN,NaN,NaN,NaN,"MARCADOR/PINCEL PARA QUADRO BRANCO, TIPO RECAR...",0,ricardo.silva@afogados.ifpe.edu.br,right_only,NaN
479,30.16.427,NaN,NaN,NaN,NaN,"MARCADOR/PINCEL PARA QUADRO BRANCO, TIPO RECAR...",0,ricardo.silva@afogados.ifpe.edu.br,right_only,NaN
480,30.24.2070,NaN,NaN,NaN,NaN,"REBITE REPUXO DE ALUMÍNIO C/ MANDRIL DE AÇO 3,...",0,ricardo.silva@afogados.ifpe.edu.br,right_only,NaN


,codigo,ed,descricao_suap,qtd_suap,qtd_lev,diferenca,acao
0,30.14.117,14 - MATERIAL EDUCATIVO E ESPORTIVO,"APITO, MATERIAL PLÁSTICO, APLICAÇÃO ESPORTE, T...",11.0,11,0.0,adequado
1,30.14.121,14 - MATERIAL EDUCATIVO E ESPORTIVO,REDE ESPORTE BASQUETE,7.0,7,0.0,adequado
2,30.14.122,14 - MATERIAL EDUCATIVO E ESPORTIVO,REDE ESPORTE FUTSAL,10.0,10,0.0,adequado
3,30.14.123,14 - MATERIAL EDUCATIVO E ESPORTIVO,REDE ESPORTE FUTEBOL DE CAMPO,3.0,3,0.0,adequado
4,30.14.13,14 - MATERIAL EDUCATIVO E ESPORTIVO,BOLA DE FUTEBOL DE CAMPO,5.0,13,-8.0,saida sem pedido
...,...,...,...,...,...,...,...
466,30.59.33,59 - MATERIAL PARA DIVULGAÇÃO,LIVRETO INSTITUCIONAL 13X22CM 16 PAGINAS CAPA ...,398.0,398,0.0,adequado
467,30.59.54,59 - MATERIAL PARA DIVULGAÇÃO,PASTA COM BOLSO INTERNO EM PAPEL CARTAO LAMINA...,500.0,500,0.0,adequado
468,30.59.55,59 - MATERIAL PARA DIVULGAÇÃO,ENVELOPE SACO PAPEL OFFSET COM ABA DE FECHAMENTO,500.0,500,0.0,adequado
469,30.59.56,59 - MATERIAL PARA DIVULGAÇÃO,BLOCO DE ANOTACOES 15 X 21CM EM PAPEL OFFSET,1000.0,1000,0.0,adequado
